In [29]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [30]:
data = pd.read_excel(io="./data/formatted/processedData201819.xlsx")
data = np.array(data)
databody = data[:, 2:].copy()
r = databody.shape[0]
c= databody.shape[1]
ind = np.random.choice(r, int(r/2), replace=False)
temp = databody[ind, 0:int(c/2)]
databody[ind, 0:int(c/2)] = databody[ind, int(c/2):]
databody[ind, int(c/2):] = temp
oury = np.ones((r, 1))
oury[ind] = 0

In [33]:
# processing input training data
input_train = tf.estimator.inputs.numpy_input_fn(
                  x = {'x': databody},
                  y = oury,
                  num_epochs = 100,
                  shuffle = False)

feature_columns = [tf.feature_column.numeric_column(key='x', shape=(databody.shape[1],))]

# makes a deep neural network with 2 hidden layers with 512 neurons each, and 10 output neurons
dnn_clf=tf.estimator.DNNClassifier(hidden_units=[512,512], n_classes=10, feature_columns=feature_columns)

# fits the DNN to the training data using the training labels
dnn_clf.fit(x=databody, y=oury, batch_size=50, steps=40000)

# evaluates the performance of the DNN on the training data
dnn_clf.evaluate(databody, oury)

AttributeError: module 'tensorflow_core.estimator' has no attribute 'inputs'